# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

Поддерживаются две операции - сложение и умножение. Применить можно так:

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [18]:
from dataclasses import dataclass
from numbers import Number
import math
from math import sin,cos,log,exp
import random
from typing import Union, Callable, List,Generator,Iterable


@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)
    
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value * o_value))
            case Number():
                return Dual(self.value / float(other), self.d / float(other))

    def __pow__(self, exponent: Union["Dual", Number]) -> "Dual":
         match exponent:
            case Dual(e_value, e_d):
                new_value = self.value ** e_value
                new_d = (self.d * e_value * self.value ** (e_value - 1)) + (e_d * new_value)
                return Dual(new_value, new_d)
            case Number():
                new_value = self.value ** float(exponent)
                new_d = self.d * float(exponent) * self.value ** (float(exponent) - 1)
                return Dual(new_value, new_d)
             
    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(o_value - self.value, o_d - self.d)
            case Number():
                return Dual(float(other) - self.value, -self.d)
             
    def __float__(self):
        return self.value

    __rmul__ = __mul__
    __radd__ = __add__

def diff(func: Callable[[float], float], xs = Iterable[float]) -> Generator[float, None,None]:
    for x in xs:
        yield func(Dual(x, 1.0)).d

# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

# значение производной в точке x = 2
xs =[-2,-1,1,2]
for test in diff(f,xs):
    print(test)

-18.0
-8.0
12.0
22.0


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [19]:
# 实现自定义的exp函数
# 计算每一项 x^i / i! 并将其添加到结果中，其中 i 逐渐增加。
# Реализовать пользовательскую функцию exp
# Вычислить каждый член x^i / i! и прибавить его к результату, где i увеличивается.
def custom_exp(x):
    result = 0
    for i in range(20):  # 计算前20个项的级数展开
        numerator = x ** i
        denominator = math.factorial(i)
        result += numerator / denominator
    return result

# 实现自定义的cos函数
# 这里使用了前10项的泰勒级数[(-1)^i*x**(2i)]/2i!来计算 cos(x)。
# 其中，sign 表示正负号，numerator 表示分子，denominator 表示分母。math.factorial(n) 函数表示计算 n 的阶乘。
# Реализация пользовательской функции cos
# Здесь для вычисления cos(x) используется ряд Тейлора [(-1)^i*x**(2i)]/2i! из первых 10 членов.
# где sign обозначает знак плюс или минус, numerator - числитель, denominator - знаменатель. math.factorial(n) функция обозначает вычисление факториала числа n.
def custom_cos(x):
    result = 0.0
    for i in range(10):
        sign = (-1) ** i
        numerator = x ** (2 * i)
        denominator = math.factorial(2 * i)
        result += sign * numerator / denominator
    return result

# 实现自定义的sin函数
# 这里使用了前10项的泰勒级数[(-1)^i*x**(2i+1)]/(2i+1)!来计算 sin(x)。
# 其中，sign 表示正负号，numerator 表示分子，denominator 表示分母。math.factorial(n) 函数表示计算 n 的阶乘。
# Реализовать пользовательскую функцию sin
def custom_sin(x):
    result = 0.0
    for i in range(10):
        sign = (-1) ** i
        numerator = x ** (2 * i + 1)
        denominator = math.factorial(2 * i + 1)
        result += sign * numerator / denominator
    return result

# 实现自定义的log函数（自然对数）
# 使用交替的正负号，计算每一项 ((-1)^(i - 1)) * (x - 1)^i / i 并将其添加到结果中，其中 i 逐渐增加。
# Реализовать пользовательскую функцию log (натуральный логарифм)
# Используя чередование знаков плюс и минус, вычислить каждый член ((-1)^(i - 1)) * (x - 1)^i / i и прибавить его к результату, где i увеличивается.
def custom_log(x):
    result = 0
    for i in range(1,1001):
        sign = (-1) ** (i -1)
        numerator = (x - 1) ** i
        result += sign * numerator / i
    return result

# 使用示例：
# Тесты
x = 2
print(exp(x))
print(f"exp({x}) = {custom_exp(x)}")
print(cos(x))
print(f"cos({x}) = {custom_cos(x)}")
print(sin(x))
print(f"sin({x}) = {custom_sin(x)}")
print(log(x))
print(f"log({x}) = {custom_log(x)}")

7.38905609893065
exp(2) = 7.3890560989301735
-0.4161468365471424
cos(2) = -0.41614683654756973
0.9092974268256817
sin(2) = 0.909297426825641
0.6931471805599453
log(2) = 0.6926474305598223


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [20]:

# f′(x)≈ f(x+ϵ)−f(x−ϵ)​/2ϵ

def numerical_derivative(func: Callable[[float], float], xs: Iterable[float], epsilon: float = 1e-6) -> Generator[float,None,None]:

    for x in xs:

        f_x_plus_epsilon = func(x + epsilon)
        f_x_minus_epsilon = func(x - epsilon)
    
        numerical_derivative = (f_x_plus_epsilon - f_x_minus_epsilon) / (2 * epsilon)
    
        yield numerical_derivative

def func(x :float) -> float:
    return x + 5*x - cos(20 * log(12- 20*x*x)) - 20*x

# 运行测试用例
# Тесты
xs= [-0.5, 0, 0.5]
for test in numerical_derivative(func,xs):
    print(test)


39.64495093899956
-14.000000000069512
-67.64495093847245


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

In [21]:
def cos(x: Union[Dual, Number]) -> Dual:
    if isinstance(x, Dual):
        return Dual(math.cos(x.value), -math.sin(x.value)*x.d)
    else:
        return Dual(math.cos(x), 0)

def sin(x: Union[Dual, Number]) -> Dual:
    if isinstance(x, Dual):
        return Dual(math.sin(x.value), math.cos(x.value)*x.d)
    else:
        return Dual(math.sin(x), 0)

def exp(x: Union[Dual, Number]) -> Dual:
    if isinstance(x, Dual):
        return Dual(math.exp(x.value), math.exp(x.value)*x.d)
    else:
        return Dual(math.exp(x), 0)

def log(x: Union[Dual, Number]) -> Dual:
    if isinstance(x, Dual):
        return Dual(math.log(x.value), x.d/x.value)
    else:
        return Dual(math.log(x), 0)


def f(x: float) -> float:
    return x + 5*x - cos(20 * log(12 - 20*x*x)) - 20*x

xs = [-0.5,-0.3,0.1,0.5]
for result in diff(f,xs):
    print(result)
# Вывести производную f при x = -0.5 39.644950968544144
# 输出f在x=-0.5处的导数  39.644950968544144


39.644950968544144
0.7243135739942659
-8.674625865643051
-67.64495096854415


In [22]:
# 创建一个空列表来存储生成的函数
# Создать пустой список для хранения сгенерированных функций
func_s = []
strs = []
xs = []

# 生成 10 个随机函数
# Генерируем 10 случайных функций
for _ in range(10):
    # 生成随机系数
    # Генерировать случайные коэффициенты
    a = random.randint(-10, 10)
    b = random.randint(-10, 10)
    c = random.randint(-10, 10)
    d = random.randint(1, 10)
    x = round(random.uniform(-0.5, 0.5),2)

    # 创建函数字符串
    # Создание строк функций
    func_str = f"lambda x: {a} * x*x + {b} * x*x*x + {c} + sin({a}*x*x) + cos({b}*x*x) + log({d}*x*x+1) + exp({a}*x*x)"

    # 使用 eval 将字符串转换为函数
    # Используйте eval для преобразования строк в функции
    func = eval(func_str)

    # 将生成的函数添加到列表中
    # Добавить сгенерированную функцию в список
    func_s.append(func)
    strs.append(func_str)
    xs.append(x)

# 测试函数
for i in range(len(func_s)):

    #生成随机x
    # x = round(random.uniform(-0.5, 0.5),2)
    print(f"Когда x = {xs[i]}, f(x) = {strs[i]}")

    #符号求导 символическая дифференциация
    dfs = diff(func_s[i],xs)
    print("символическая дифференциация:",next(dfs))

    #数值求导 численное дифференцирование
    res = numerical_derivative(func_s[i], xs)   
    print("численное дифференцирование:",next(res).value)

Когда x = 0.45, f(x) = lambda x: -2 * x*x + -3 * x*x*x + 4 + sin(-2*x*x) + cos(-3*x*x) + log(6*x*x+1) + exp(-2*x*x)
символическая дифференциация: -5.580724014226629
численное дифференцирование: -5.580724014464522
Когда x = 0.01, f(x) = lambda x: 10 * x*x + -5 * x*x*x + -10 + sin(10*x*x) + cos(-5*x*x) + log(8*x*x+1) + exp(10*x*x)
символическая дифференциация: 69.13015349874031
численное дифференцирование: 69.13015349807594
Когда x = 0.27, f(x) = lambda x: -7 * x*x + -6 * x*x*x + 3 + sin(-7*x*x) + cos(-6*x*x) + log(5*x*x+1) + exp(-7*x*x)
символическая дифференциация: -15.259368710693883
численное дифференцирование: -15.259368710385246
Когда x = -0.42, f(x) = lambda x: -4 * x*x + -9 * x*x*x + 4 + sin(-4*x*x) + cos(-9*x*x) + log(5*x*x+1) + exp(-4*x*x)
символическая дифференциация: -18.75992366660356
численное дифференцирование: -18.75992366606738
Когда x = 0.16, f(x) = lambda x: -3 * x*x + -1 * x*x*x + -6 + sin(-3*x*x) + cos(-1*x*x) + log(3*x*x+1) + exp(-3*x*x)
символическая дифференциация

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [23]:

def diffs(func: Callable[..., Dual]) -> Callable[..., List[float]]:
    def wrapper(*args, **kwargs):

        for i in range(len(args)):

            dual_args = [Dual(arg, 1 if j == i else 0) for j, arg in enumerate(args)]
            yield func(*dual_args, **kwargs).d

    return wrapper

def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y

f_diffs = diffs(f)

# 计算在参数 x = 10, y = 10, z = 10 处关于 x、y 和 z 的导数值
# Вычислить производные от x, y и z при x = 10, y = 10, z = 10
for result in f_diffs(10,10,10):
    print(result)
# 输出结果应为 [10.0, 5.0, 1.0]
# На выходе должно получиться [10.0, 5.0, 1.0].

10.0
5.0
1.0
